In [2]:
import pandas as pd
import ast

## Preprocess Diyetkolik 

In [3]:
df = pd.read_excel("data/diyetkolik_recipes.xlsx", index_col=0)

In [3]:
def parse_string_as_list(string):
    try:
        return ast.literal_eval(string)
    except (ValueError, SyntaxError):
        return string

In [4]:
df.Ingredients= df.Ingredients.apply(parse_string_as_list)

In [5]:
ing_counts = dict()

for row in df.Ingredients:
    for ing in row:
        if ing not in ing_counts.keys():
            ing_counts[ing] = 1
        else:
            ing_counts[ing] += 1     

In [6]:
for ing, count in list(ing_counts.items()):
    if count < 15:
        del ing_counts[ing]

In [7]:
new_df = pd.DataFrame(0, index=df.index, columns=ing_counts.keys())

In [8]:
for index, recipe in df.iterrows():
    ingredients = recipe["Ingredients"]
    for ingredient in ingredients:
        if ingredient in new_df.columns.values:
            new_df.at[index,ingredient] = 1

In [9]:
new_df = new_df[ing_counts.keys()][0:1325]

,allspice,almond,amaretto,anise,apple,applesauce,apricot,artichoke,arugula,asparagus,...,watercress,watermelon,wheat,whip,whiskey,wine,wrapper,yeast,yoghurt,yogurt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71617,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Cluster With FINCH

In [5]:
from finch import FINCH
df = pd.read_excel("data/synthetic_dataset.xlsx", index_col=0)

c, num_clust, req_c = FINCH(df)

/Users/emrekuru/Downloads/Contrastive-Learning/env/lib/python3.11/site-packages/finch/finch.py:13: UserWarning: pynndescent not installed: No module named 'pynndescent'
  warnings.warn('pynndescent not installed: {}'.format(e))


Partition 0: 10002 clusters
Partition 1: 1275 clusters
Partition 2: 187 clusters
Partition 3: 27 clusters
Partition 4: 5 clusters


In [6]:
num_clust

[10002, 1275, 187, 27, 5]

In [10]:
pd.DataFrame(c)[4]

0        0
1        0
2        0
3        1
4        0
        ..
69260    3
69261    0
69262    4
69263    1
69264    4
Name: 4, Length: 69265, dtype: int32